In [1]:
from carsus import init_db
session = init_db("sqlite://")
session.commit()

Initializing the database
Ingesting basic atomic data


In [2]:
from carsus.io.nist import NISTWeightsCompIngester
ingester = NISTWeightsCompIngester()
ingester.download()
ingester.ingest(session)

Ingesting atomic weights


In [3]:
from carsus.io.output import AtomsDataset, Dataset

# Create a dataset and load with the atoms data from the database. 

atoms = AtomsDataset()
atoms.load_sql(session)
print atoms.data.head(5)


              symbol       name  group  period
atomic_number                                 
1                  H   Hydrogen      1       1
2                 He     Helium     18       1
3                 Li    Lithium      1       2
4                 Be  Beryllium      2       2
5                  B      Boron     13       2


In [4]:
# Include atomic weights into the dataset

atoms2 = AtomsDataset()
atoms2.load_sql(session, load_atomic_weights=True)
print atoms2.data.head(5)

              symbol       name  group  period  atomic_weight_value  \
atomic_number                                                         
1                  H   Hydrogen      1       1             1.007975   
2                 He     Helium     18       1             4.002602   
3                 Li    Lithium      1       2             6.967500   
4                 Be  Beryllium      2       2             9.012183   
5                  B      Boron     13       2            10.813500   

               atomic_weight_uncert atomic_weight_data_source  
atomic_number                                                  
1                      1.350000e-04                      nist  
2                      2.000000e-06                      nist  
3                      2.950000e-02                      nist  
4                      5.000000e-07                      nist  
5                      7.500000e-03                      nist  


In [5]:
# Lets put out a table with z, weight in solar masses, z**2 + 5

# First we need to write a query
# Unit conversion is done on the DB side!
from carsus.model import Atom, AtomicWeight
from astropy import units as u
q = session.query(Atom.atomic_number.label("z"),
                  AtomicWeight.quantity.convert_to(u.solMass).value.label("weight_solMass")).\
                  join(Atom.quantities.of_type(AtomicWeight))

# Then we use pandas to read the query into a DataFrame
from pandas import read_sql_query
df = read_sql_query(q.selectable, session.bind)

# Once we have the data we can compute things
df["z**2+5"] = df["z"]**2 + 5

print df.head(5)

   z  weight_solMass  z**2+5
0  1    8.414768e-58       6
1  2    3.341449e-57       9
2  3    5.816602e-57      14
3  4    7.523543e-57      21
4  5    9.027316e-57      30


In [6]:
# To store this DataFrame in HDF format we need to create a Dataset object

foo_ds = Dataset("foo", df)

In [7]:
# create_hdf() is used to put several datasets into an HDF file. Datasets' names are used as the identifiers in the file.

from carsus.io.output import create_hdf
create_hdf("store.h5", datasets=[atoms, foo_ds])


/home/misha/.virtualenvs/carsus/lib/python2.7/site-packages/ipykernel/__main__.py:4: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->unicode,key->block1_values] [items->['symbol', 'name']]



In [9]:
from pandas import read_hdf
atoms = read_hdf("store.h5", "foo")
print atoms.head(5)

   z  weight_solMass  z**2+5
0  1    8.414768e-58       6
1  2    3.341449e-57       9
2  3    5.816602e-57      14
3  4    7.523543e-57      21
4  5    9.027316e-57      30
